In [1]:
import cv2
import numpy as np
from datetime import datetime
import time
from collections import deque

In [2]:
def calculate_angle(a, b, c):
    """Calculate angle between three points"""
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
    return angle

def extract_body_silhouette(frame, bg_subtractor):
    """Extract human body silhouette"""
    # Apply background subtraction
    fg_mask = bg_subtractor.apply(frame)
    
    # Denoise and refine mask
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))
    fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_CLOSE, kernel)
    fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_OPEN, kernel)
    
    # Find largest contour (body)
    contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if not contours:
        return None, fg_mask
    
    # Get largest contour
    body_contour = max(contours, key=cv2.contourArea)
    
    if cv2.contourArea(body_contour) < 5000:  # Filter small detections
        return None, fg_mask
    
    return body_contour, fg_mask

def approximate_skeleton(contour, frame_shape):
    """Approximate body skeleton from contour"""
    if contour is None:
        return None
    
    h, w = frame_shape[:2]
    
    # Get bounding rectangle and moments
    x, y, rect_w, rect_h = cv2.boundingRect(contour)
    M = cv2.moments(contour)
    
    if M["m00"] == 0:
        return None
    
    # Calculate center of mass (torso center)
    cx = int(M["m10"] / M["m00"])
    cy = int(M["m01"] / M["m00"])
    
    # Approximate key points based on body proportions
    skeleton = {}
    
    # Head (top of bounding box)
    skeleton['head'] = (cx, y + int(rect_h * 0.1))
    
    # Neck
    skeleton['neck'] = (cx, y + int(rect_h * 0.15))
    
    # Shoulders (width proportional to body)
    shoulder_y = y + int(rect_h * 0.2)
    shoulder_offset = int(rect_w * 0.35)
    skeleton['left_shoulder'] = (cx - shoulder_offset, shoulder_y)
    skeleton['right_shoulder'] = (cx + shoulder_offset, shoulder_y)
    
    # Torso center
    skeleton['torso'] = (cx, cy)
    
    # Hips
    hip_y = y + int(rect_h * 0.55)
    hip_offset = int(rect_w * 0.25)
    skeleton['left_hip'] = (cx - hip_offset, hip_y)
    skeleton['right_hip'] = (cx + hip_offset, hip_y)
    
    # Find extremity points (hands and feet)
    # Get convex hull and defects
    hull = cv2.convexHull(contour, returnPoints=False)
    if len(hull) > 3:
        defects = cv2.convexityDefects(contour, hull)
        
        if defects is not None:
            extremities = []
            for i in range(defects.shape[0]):
                s, e, f, d = defects[i, 0]
                start = tuple(contour[s][0])
                extremities.append(start)
            
            # Sort extremities by Y coordinate (top to bottom)
            extremities = sorted(extremities, key=lambda p: p[1])
            
            # Classify extremities as hands or feet based on Y position
            mid_y = y + rect_h // 2
            hands = [p for p in extremities if p[1] < mid_y]
            feet = [p for p in extremities if p[1] > mid_y]
            
            # Assign hands (left/right based on X)
            if len(hands) >= 2:
                hands = sorted(hands, key=lambda p: p[0])
                skeleton['left_hand'] = hands[0]
                skeleton['right_hand'] = hands[-1]
            
            # Assign feet (left/right based on X)
            if len(feet) >= 2:
                feet = sorted(feet, key=lambda p: p[0])
                skeleton['left_foot'] = feet[0]
                skeleton['right_foot'] = feet[-1]
    
    # Estimate elbows (midpoint between shoulder and hand)
    if 'left_hand' in skeleton:
        skeleton['left_elbow'] = (
            (skeleton['left_shoulder'][0] + skeleton['left_hand'][0]) // 2,
            (skeleton['left_shoulder'][1] + skeleton['left_hand'][1]) // 2
        )
    if 'right_hand' in skeleton:
        skeleton['right_elbow'] = (
            (skeleton['right_shoulder'][0] + skeleton['right_hand'][0]) // 2,
            (skeleton['right_shoulder'][1] + skeleton['right_hand'][1]) // 2
        )
    
    # Estimate knees (midpoint between hip and foot)
    if 'left_foot' in skeleton:
        skeleton['left_knee'] = (
            (skeleton['left_hip'][0] + skeleton['left_foot'][0]) // 2,
            (skeleton['left_hip'][1] + skeleton['left_foot'][1]) // 2
        )
    if 'right_foot' in skeleton:
        skeleton['right_knee'] = (
            (skeleton['right_hip'][0] + skeleton['right_foot'][0]) // 2,
            (skeleton['right_hip'][1] + skeleton['right_foot'][1]) // 2
        )
    
    return skeleton

In [3]:
def draw_skeleton(frame, skeleton):
    """Draw skeleton connections on frame"""
    if skeleton is None:
        return frame
    
    # Define skeleton connections
    connections = [
        ('head', 'neck'),
        ('neck', 'left_shoulder'),
        ('neck', 'right_shoulder'),
        ('left_shoulder', 'left_elbow'),
        ('left_elbow', 'left_hand'),
        ('right_shoulder', 'right_elbow'),
        ('right_elbow', 'right_hand'),
        ('neck', 'torso'),
        ('torso', 'left_hip'),
        ('torso', 'right_hip'),
        ('left_hip', 'right_hip'),
        ('left_hip', 'left_knee'),
        ('left_knee', 'left_foot'),
        ('right_hip', 'right_knee'),
        ('right_knee', 'right_foot'),
    ]
    
    # Color coding
    colors = {
        'head': (255, 255, 0),      # Yellow
        'neck': (255, 255, 0),
        'torso': (255, 0, 0),        # Blue
        'left_shoulder': (255, 0, 0),
        'right_shoulder': (255, 0, 0),
        'left_hip': (255, 0, 0),
        'right_hip': (255, 0, 0),
        'left_elbow': (0, 255, 0),   # Green (arms)
        'right_elbow': (0, 255, 0),
        'left_hand': (0, 255, 0),
        'right_hand': (0, 255, 0),
        'left_knee': (255, 0, 255),  # Magenta (legs)
        'right_knee': (255, 0, 255),
        'left_foot': (255, 0, 255),
        'right_foot': (255, 0, 255),
    }
    
    # Draw connections
    for start_joint, end_joint in connections:
        if start_joint in skeleton and end_joint in skeleton:
            cv2.line(frame, skeleton[start_joint], skeleton[end_joint], (0, 255, 255), 3)
    
    # Draw joints
    for joint_name, joint_pos in skeleton.items():
        color = colors.get(joint_name, (255, 255, 255))
        cv2.circle(frame, joint_pos, 6, color, -1)
        cv2.circle(frame, joint_pos, 7, (0, 0, 0), 1)
    
    return frame

In [4]:
def body_pose_tracker(camera_id=0, show_contour=True):
    """
    Real-time body pose tracking using contour analysis
    
    Args:
        camera_id: Camera index (default: 0)
        show_contour: Show body contour
    
    Controls:
        Q - Quit
        S - Save frame
        C - Toggle contour
        B - Recalibrate background
    """
    cap = cv2.VideoCapture(camera_id)
    if not cap.isOpened():
        return
    
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
    cap.set(cv2.CAP_PROP_FPS, 30)
    
    bg_subtractor = cv2.createBackgroundSubtractorMOG2(
        history=500, varThreshold=25, detectShadows=False
    )
    
    prev_time = 0
    frame_count = 0
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        curr_time = time.time()
        fps = 1 / (curr_time - prev_time) if prev_time > 0 else 0
        prev_time = curr_time
        frame_count += 1
        
        body_contour, fg_mask = extract_body_silhouette(frame, bg_subtractor)
        skeleton = approximate_skeleton(body_contour, frame.shape)
        
        if show_contour and body_contour is not None:
            cv2.drawContours(frame, [body_contour], -1, (0, 255, 0), 2)
        
        if skeleton:
            frame = draw_skeleton(frame, skeleton)
        
        cv2.putText(frame, f'FPS: {fps:.1f}', (20, 40),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
        
        status = "Tracking" if skeleton else "No body detected"
        status_color = (0, 255, 0) if skeleton else (0, 165, 255)
        cv2.putText(frame, status, (20, 75),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, status_color, 2)
        
        cv2.putText(frame, "Q:Quit | S:Save | C:Contour | B:Recalibrate", 
                   (10, frame.shape[0] - 20),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        
        if frame_count < 90:
            cv2.putText(frame, "CALIBRATING...", 
                       (frame.shape[1]//2 - 150, frame.shape[0]//2),
                       cv2.FONT_HERSHEY_DUPLEX, 1.0, (0, 0, 255), 3)
        
        cv2.imshow('Body Pose Tracker', frame)
        
        key = cv2.waitKey(1) & 0xFF
        if key in (ord('q'), ord('Q')):
            break
        elif key in (ord('s'), ord('S')):
            img_name = f'pose_{datetime.now().strftime("%Y%m%d_%H%M%S")}.jpg'
            cv2.imwrite(img_name, frame)
        elif key in (ord('c'), ord('C')):
            show_contour = not show_contour
        elif key in (ord('b'), ord('B')):
            bg_subtractor = cv2.createBackgroundSubtractorMOG2(
                history=500, varThreshold=25, detectShadows=False
            )
    
    cap.release()
    cv2.destroyAllWindows()


In [5]:
body_pose_tracker(camera_id=0, show_contour=True)
